

# Objetivo

En este trabajo vamos a analizar los datos epidemiológicos generados por el grupo de trabajo [#escovid19data](https://github.com/montera34/escovid19data). Siguiendo los procesos estándares descargaremos los datos desde gitub, analizaremos los campos y prepararemos una serie de gráficas como ayuda para entender como estñá progresando la enfermedad.


@author: Manuel H. Arias 

@Twitter: @walyt

@mail: mharias@me.com



## Importación de librerias estándares para DAE

In [1]:
import pandas as pd

import matplotlib.pyplot as plt

from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.colors as colors

import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

import urllib.request

import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox

## Descargamos la información

Hacemos en primer lugar una actualización de parámetros y preparación de variables que necesitaremos durante el ejercicio

In [2]:
pd.options.display.max_rows = 999 #Variable de contexto para permitir la presentación de datos por pantalla
pd.set_option('display.max_columns', None)

In [3]:
#fuente de pdatos
data_path = '/Users/mharias/documents/proyectos/covid/sanidad_hospital/data/datos_sanidad_hosp.csv'
img_path = '/Users/mharias/documents/proyectos/covid/sanidad_hospital/img/'

In [4]:
hashtag_fuente='#escovid19data' #hashtag de la fuente para los créditos

Leemos los datos en un `pandas`

In [5]:
df = pd.read_csv(data_path)

veamos una rápida descripción de la información:

In [6]:
df.describe()

,Hosp_Totales,Hosp_actual,UCI_actual,Ingresos_24horas,Altas_24horas
count,8455.000000,2945.000000,2945.000000,2945.000000,2945.000000
mean,158.034793,264.460576,158.691681,103.791171,97.557895
std,541.942406,280.060555,187.133069,130.765688,131.668580
min,1.000000,1.000000,0.000000,0.000000,0.000000
25%,2.655000,4.488000,38.000000,19.000000,16.000000
50%,7.776000,182.000000,84.000000,51.000000,44.000000
75%,29.449500,469.000000,207.000000,133.000000,122.000000
max,4991.000000,996.000000,824.000000,684.000000,729.000000


y un muestreo de valores y de algunas columnas de interés:

In [7]:
df.head()

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
0,2020-04-27,Madrid,39.472,,3.309,,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-27,País Vasco,6.457,31,538,5,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-27,Navarra,1.942,0,130,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-27,Murcia,627.000,0,106,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-27,Melilla,44.000,0,3,0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.tail()

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
8450,2021-02-16,Madrid,85.342,692,6.141,59,3.794,"21,57%",690.0,"49,29%",286.0,506.0
8451,2021-02-16,Melilla,472.000,15,71,2,33.000,"18,13%",6.0,"35,29%",2.0,5.0
8452,2021-02-16,Murcia,8.295,121,1.057,12,457.000,"11,08%",119.0,"24,29%",48.0,69.0
8453,2021-02-16,La Rioja,3.132,42,350,3,133.000,"15,89%",46.0,"52,87%",4.0,33.0
8454,2021-02-16,Navarra,5.071,54,436,5,125.000,"6,44%",24.0,"18,32%",10.0,16.0


In [9]:
def formato_(x):
    if x is np.NaN:
        return x
    else:
        return float(x[:-2].replace(',','.'))
    

In [10]:
df['%CamasUCI'] = df['%CamasUCI'].apply(formato_)
df['%CamasCovid'] = df['%CamasCovid'].apply(formato_)

In [11]:
comunidades = sorted(df['ccaa'].unique())

Vamos a formatear correctamente la columna `Date`, para posteriormente ordenar el $pandas$ por fecha creciente:

In [12]:
df['fecha']=pd.to_datetime(df['fecha'],format='%Y-%m-%d')

In [13]:
df.sort_values(['fecha']).tail()

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
8438,2021-02-16,Andalucía,35.631,791,3.408,33,3.23,17.80,644.0,33.33,284.0,483.0
8437,2021-02-16,País Vasco,11.879,29,993,7,707.00,14.14,151.0,33.33,72.0,132.0
8453,2021-02-16,La Rioja,3.132,42,350,3,133.00,15.89,46.0,52.87,4.0,33.0
8444,2021-02-16,Castilla La Mancha,17.230,56,1.188,6,825.00,15.38,202.0,41.91,34.0,108.0
8454,2021-02-16,Navarra,5.071,54,436,5,125.00,6.44,24.0,18.32,10.0,16.0


In [14]:
df.columns

Index(['fecha', 'ccaa', 'Hosp_Totales', 'Hosp_7días', 'UCI_Totales',
       'UCI_7dias', 'Hosp_actual', '%CamasCovid', 'UCI_actual', '%CamasUCI',
       'Ingresos_24horas', 'Altas_24horas'],
      dtype='object')

In [15]:
campos = {'Almería':'daily_cases', 'Cádiz':'daily_cases', 'Córdoba':'daily_cases',
          'Granada':'daily_cases', 'Huelva':'daily_cases', 'Jaén':'daily_cases',
       'Málaga':'daily_cases', 'Sevilla':'daily_cases', 'Huesca':'casos_nuevos_aragon',
          'Teruel':'casos_nuevos_aragon', 'Zaragoza':'casos_nuevos_aragon', 
          'Asturias':'PCR','Balears, Illes':'PCR', 'Palmas, Las':'daily_cases', 
          'Santa Cruz de Tenerife':'daily_cases','Cantabria':'PCR',
          'Albacete':'daily_cases_PCR', 'Ciudad Real':'daily_cases_PCR',
          'Cuenca':'daily_cases_PCR', 'Guadalajara':'daily_cases_PCR',
       'Toledo':'daily_cases_PCR', 'Ávila':'daily_cases_PCR', 'Burgos':'daily_cases_PCR',
          'León':'daily_cases_PCR', 'Palencia':'daily_cases_PCR', 'Salamanca':'daily_cases_PCR',
       'Segovia':'daily_cases_PCR', 'Soria':'daily_cases_PCR', 'Valladolid':'daily_cases_PCR',
          'Zamora':'daily_cases_PCR', 'Barcelona':'daily_cases', 'Girona':'daily_cases',
       'Lleida':'daily_cases', 'Tarragona':'daily_cases', 'Ceuta':'new_cases',
          'Alicante/Alacant':'casos_nuevos_valencia','Castellón/Castelló':'casos_nuevos_valencia',
          'Valencia/València':'casos_nuevos_valencia','Badajoz':'new_cases', 'Cáceres':'new_cases',
       'Coruña, A':'daily_cases', 'Lugo':'daily_cases', 'Ourense':'daily_cases', 
          'Pontevedra':'daily_cases', 'Madrid':'daily_cases_PCR', 'Melilla':'PCR',
       'Murcia':'daily_cases', 'Navarra':'PCR', 'Araba/Álava':'casos_nuevos_paisvasco',
          'Bizkaia':'casos_nuevos_paisvasco', 'Gipuzkoa':'casos_nuevos_paisvasco',
       'Rioja, La':'PCR'}

# Gráficos

 A continuación vamos a crear un gráfico múltiple que nos permita visualizar cada una de las columnas con datos numéricos. Ello nos permitirá

Preparemos un gráfico tipo [Facetgrid](https://seaborn.pydata.org/generated/seaborn.FacetGrid.html), al que añadiremos tantos gráficos como provincias, presentando una de las columnas de datos. Elegimos nº de hospitalizados por 100.000 habitantes. Dejo otras claves con `#` para que fácilmente se puedan generar. Este ejemplo esta basado en el código del link anterior.

In [16]:
df.iloc[1]['fecha'].year

2020

In [17]:
df['fecha']

0      2020-04-27
1      2020-04-27
2      2020-04-27
3      2020-04-27
4      2020-04-27
          ...    
8450   2021-02-16
8451   2021-02-16
8452   2021-02-16
8453   2021-02-16
8454   2021-02-16
Name: fecha, Length: 8455, dtype: datetime64[ns]

In [ ]:
# Creamos la figura desde Matplotlib
#plt.figure()

# Initialize the FacetGrid object
#pal = sns.cubehelix_palette(52, rot=-.25, light=.7)

with sns.plotting_context('paper', font_scale = 1.3):
    g = sns.FacetGrid(df,col='ccaa',hue='ccaa',col_wrap=3,aspect=3, 
                      margin_titles=False,height=1.5,sharey=True)

# Título

titulo = 'EVOLUCION DE OCUPACION DE CAMAS HOSPITAL Y UCI POR COVID \n Fuente Informes diarios Sanidad: {}'.format('16-feb')
color_titulos = 'navy'
color_uci='red'
color_covid='navy'

#g.fig.text(1, 0,'Fuente: {}'.format(hashtag_fuente), fontsize=10,horizontalalignment='right') #add text
#g.fig.text(1,0.01,'Código:{}'.format(github_url),
          # fontsize=10,horizontalalignment='right')
# Draw the densities in a few steps

XmajorFmt = DateFormatter('%-d-%-b')
fmt = '%.0f%%' # Format you want the ticks, e.g. '40%'
yticks = ticker.FormatStrFormatter(fmt)
def grafica(x, color, label):
    ax1 = plt.gca()
    ax1.xaxis_date()
    df_datos = df.loc[df['ccaa'] == label].sort_values(['fecha'])
   
    #ax1.fill_between(datos['date'].unique(), datos[AI],color=color_diarios)
    ax1.text(0.5, .8, label, fontweight="bold", color=color_titulos,
            ha="center", va="center", size=16,transform=ax1.transAxes)
    ax1.set_ylim(0,75)
    ax1.set_yticks([0,25,50,75])
    
    ax1.yaxis.set_major_formatter(yticks)
    #ax1.set_yticks([]
    ax1.yaxis.set_tick_params(labelsize=12,width=0,labelcolor=color_covid)
    #ax1.yaxis.set_major_locator(plt.MaxNLocator(5))
    ax1.xaxis.set_tick_params(labelsize=12,width=0,labelcolor=color_covid,rotation=45)
    ax1.xaxis.set_major_formatter(XmajorFmt)
    #ax1.set_xticks([])
    ax1.grid(True,axis='both',which='both')
    ax1.plot(df_datos['fecha'], df_datos['%CamasCovid'],lw=3,color=color_covid,label='A')
    
    
    ax2 = ax1.twinx()
    ax2.plot(df_datos['fecha'], df_datos['%CamasUCI'], lw=3, color=color_uci,label='B')
    ax2.yaxis.set_tick_params(labelsize=12,width=0,labelcolor=color_uci,pad=-10)
    ax2.yaxis.set_major_formatter(yticks)
    #ax2.set_ylim(0,4)
    #ax2.set_yticks([1])
    ax2.xaxis.set_major_formatter(XmajorFmt)
    
    ax2.set_ylim(0,75)
    ax2.set_yticks([0,25,50,75])
    ax2.set_yticks([])
    #ax2.xaxis.set_tick_params(labelsize=20,labelcolor='navy')
    #ax2.xaxis.set_major_locator(DayLocator([1]))
    #ax2.legend()
    
g.map(grafica,'ccaa')

#plt.legend()
g.set_titles("")
#g.set(yticks=[])
g.set(xlabel='')

g.despine(bottom=False, left=True)
g.fig.suptitle('{}'.format(titulo),fontsize=24,color='navy')
g.fig.text(x=.35,y=.05,s='ROJO: CAMAS UCI COVID vs UCI TOTALES',color='red',fontsize=26)
g.fig.text(x=.35,y=.1,s='AZUL: CAMAS HOSP COVID vs CAMAS TOTALES',color='navy',fontsize=26)

g.fig.subplots_adjust(top=.9,wspace=0.25,hspace=0.4)
#plt.tight_layout()
plt.savefig(img_path+'uci_hosp_comunidades.png',dpi=288)  

plt.show()
